In [10]:
                                                   
####################################### PROGRAMME RESEAU DE NEURONE ########################################

######### IMPORT ##########
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 
import pandas 
import os
import statistics
import math 
import datetime 
import scipy
from scipy.signal import savgol_filter
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.constraints import max_norm
from keras.layers import Dropout
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import learning_curve 
from sklearn.feature_selection import SelectKBest, f_classif 
from sklearn.pipeline import make_pipeline 
from sklearn.model_selection import RandomizedSearchCV


######## OPTION PREPROCSESSING ######## 

PREPROCESSING = "Oui"

########CHARGE DU DATAFRAME CORRESPONDANT A LA DATABASE DE LA TRANSMITANC ET CREATION DU TRAINSET ET DU TESTSET #########

def dataframe ():
    #print(os.listdir())
    no = pd.read_excel ("Nombre_d'onde.xlsx")
    data = pd.read_excel("DATABASE_Sapin_Epicea_Ref1'.xlsx")
    data = data.set_index("File Name")
    #data       #Retirer le # pour voir le dataset 
    print ('Data set:', data.shape)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

    print ("Train set:"  ,trainset.shape)
    print("Test set:", testset.shape)

    #trainset['Essence'].value_counts()
    #testset ['Essence'].value_counts()

    #trainset      #Retirer le # pour voir le trainset ou le testset 
    #testset 
    return data, no, trainset, testset

######### FONCTIONS DE PRE-PROCESSING DES DONNEES #########

#ENCODAGE DES VARIABLES

def encodage (data):
    code1 = {'Bout':0,
           'Chant':1,
           'Face':2}
    data['FACES'] = data['FACES'].map(code1)
    code2 = {'Epicea':0,
            'Sapin':1}
    data['Essence'] = data ['Essence'].map(code2)
    
    return data
#encodage (data)


#APPLICATION DE LA FONCTION -LOG POUR PASSER DE LA TRANSMITANCE A L'ABSORBANCE: 
def trans_abs(data):
    abs = pd.DataFrame((np.log10(data.iloc[0: , 2:].values))*-1)
    data[data.columns[2:]] = abs[abs.columns[0:2151]].values 

    return data
#trans_abs(data)

#SG1

def SG1(data, no): 
    for m in range (0 , len(data)):
        x = no.values
        y = data.iloc[m , 2: ]
        yhat = scipy.signal.savgol_filter(y, 5, 3)
        
        yp = np.zeros(2150)
        
        for i in range (0, 2150):
            yp[i] = (yhat[i+1] - yhat[i]) / (x[i+1] - x[i])   
            
        data.iloc [m, 2:2152] = yp     
    return data
# SG1 (data)

#NORMALISATION DES DONNEES DE LA DATABASE:
def normalisation (data): 
    for i in range (0 , len(data)):
        normalisation = (data.iloc[i , 2: ] - (statistics.mean(data.iloc[i , 2: ]))) / (statistics.stdev(data.iloc[i , 2: ]))
        data.iloc[i, 2: ] = normalisation.values
    return data
#normalisation (data)  


#IMPUTER 2
def imputer (data):
    data = data.drop(data.columns[2152], axis=1)
    data = data.drop(data.columns[1420:1480], axis=1)
    data = data.drop(data.columns[2:902], axis=1)    
 
    return data
#imputer (data)

#INVERSE L'ORDRE DES NOMBRES D'ondes 
def invers_axes(data):
    data = data.reindex(columns=data.columns[: :-1])
    return (data)
#invers_axes(data)

######## FONCTION QUI FAIT APPELLE AU DIFFERENTS PREPROCESSING ####### 

def preprocessing (data, no):
    data = encodage (data)
    print ("encodage terminé")
    data = trans_abs(data)
    print("transmitance to absorbance terminé")
    data = SG1(data, no)
    print ("SG1 terminé")
    data = normalisation (data)
    print ("normalisation terminée")
    data = imputer (data)
    print ("imputation terminée")
    data = invers_axes(data)
    print ("inversion des axes terminée")
    
    X = data.drop('Essence', axis=1)
    y = data['Essence']
    
    print(y.value_counts())
    
    return X, y 

######## SAUVEGARDE DES DONNEES PRE-TRAITEES #######

def sauvegarde (X_train, y_train, X_test, y_test):
    X_train.to_pickle('X_trains.pkl')
    y_train.to_pickle('y_trains.pkl')
    X_test.to_pickle('X_test.pkl')
    y_test.to_pickle('y_test.pkl')
    
######## LECTURE DES DONNEES PRE-TRAITEES ########

def load ():
    X_train = pd.read_pickle('X_trains.pkl')
    y_train = pd.read_pickle('y_trains.pkl')
    X_test = pd.read_pickle('X_test.pkl')
    y_test = pd.read_pickle('y_test.pkl')
    
    print ("X_train:", X_train.shape)
    print ("y_train:", y_train.shape)
    print("X_test:", X_test.shape)
    print("y_test:", y_test.shape)
    
    
    return (X_train, y_train, X_test, y_test) 

######### PROCEDURE D'EVALUATION DU RESEAU DE NEURONE ######## 

log_dir = "logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

def evaluation_dl (model, X_train, y_train, X_test, y_test): 
    model.fit(X=X_train, y=y_train , kerasclassifier__callbacks=[tensorboard_callback])
    
    ypred_train = model.predict(X_train)
    ypred_train = [1 if y>= 0.5 else 0 for y in ypred_train]
    ypred_test = model.predict(X_test)
    ypred_test = [1 if y>= 0.5 else 0 for y in ypred_test]
    
#I-Evaluation du model sur le train data set
    
    #Matrice de confusion
    print('Confusion Matrix trainset: \n', confusion_matrix(y_train, ypred_train))
    
    #Tableau de Score 
    print (classification_report(y_train, ypred_train))
    print ('Accuracy trainset: ', accuracy_score(y_train, ypred_train))
    
    
#II- Evaluation du model sur le test data set
    #Matrice de confusion
    print('Confusion Matrix testset: \n', confusion_matrix(y_test, ypred_test))
    
    #Tableau de Score 
    print (classification_report(y_test, ypred_test))
    print ('Accuracy testset: ', accuracy_score(y_test, ypred_test))
    

######## SELECTION DES VARIABLES ########

selection =  SelectKBest(f_classif, k=10)
    
######## LE MODEL DE DEEP LEARNING ########

def create_model ( activation = 'tanh', dropout_rate = 0.3, init_mode = 'he_normal', weight_constraint =25,  optimizer= 'Nadam', learn_rate = 0.1 , momentum=0, input_dim = 10, neurons=30):
    
    #Creat the model 
    md = Sequential ()
    md.add(Dense (neurons,
                 input_dim=input_dim, kernel_initializer=init_mode, activation=activation, kernel_constraint=max_norm(weight_constraint)))
    md.add(Dropout(dropout_rate))
    md.add(Dense(1, kernel_initializer=init_mode, activation= 'sigmoid'))
    
    #Compile model 
    
    md.compile(loss= 'binary_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])
    
    return md 

#model 
model = make_pipeline(selection, KerasClassifier(build_fn= create_model, batch_size=1000, epochs=10))

######## REGLAGE DES HYPERS PARAMS #########

def best_hyper_params (X_train, y_train, X_test, y_test): 

###### Choisissez les plages d'hyperparamètres à régler 
    activation = ['sigmoid', 'relu', 'tanh', 'hard_sigmoid', 'linear']
    momemtum = [0.0, 0.1, 0.4, 0.6, 0.8, 0.9]
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    dropout_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    weight_constraint= [1, 5, 10, 15, 20, 25, 30]
    neurons = [1, 5, 10, 15, 20, 25, 30]
    init =['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    optimizer = ['SGD' , 'RMSprop', 'Adagrad', 'Adadelta', 'Adam' , 'Adamax' , 'Nadam']

###### Choisissez les plages d'hyperparamètres à régler 
    epochs = [3000]   
    batch_size = [1000 , 2000]
    param_grid = dict(kerasclassifier__epochs=epochs)  # Ajoutez d'autres paramètres à regler (ex: kerasclassifier__activation, kerasclassifier__momemtum ...)
    
######
    grid = RandomizedSearchCV(model, param_grid, scoring = 'accuracy', cv=4, n_iter=50)
    grid_result = grid.fit(X_train, y_train)

######
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
######
    y_pred = grid.predict(X_test)
    
    return grid.best_estimator_

######### MAIN FUNCTION #########

def main ():
    if PREPROCESSING == "Oui":
        data, no, trainset, testset = dataframe ()
        X_train, y_train = preprocessing(trainset, no)
        X_test, y_test = preprocessing(testset, no)
        #sauvegarde (X_train, y_train, X_test, y_test)
        print (evaluation_dl (best_hyper_params (X_train, y_train, X_test, y_test),X_train, y_train, X_test, y_test ))
        
    else: 
        X_train, y_train, X_test, y_test = load () 
        print (evaluation_dl (best_hyper_params (X_train, y_train, X_test, y_test),X_train, y_train, X_test, y_test ))

if __name__== "__main__" : 
    main()


Data set: (286, 2153)
Train set: (228, 2153)
Test set: (58, 2153)
encodage terminé


C:\Users\louis\anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\louis\anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\louis\anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

transmitance to absorbance terminé
SG1 terminé
normalisation terminée
imputation terminée
inversion des axes terminée
0    115
1    113
Name: Essence, dtype: int64
encodage terminé
transmitance to absorbance terminé
SG1 terminé
normalisation terminée
imputation terminée
inversion des axes terminée
1    32
0    26
Name: Essence, dtype: int64


C:\Users\louis\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=50. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Epoch 1/3000
171/171 [==============================] - 0s 1ms/step - loss: 0.6936 - accuracy: 0.5029
Epoch 2/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6921 - accuracy: 0.5029
Epoch 3/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6923 - accuracy: 0.5088
Epoch 4/3000
171/171 [==============================] - 0s 17us/step - loss: 0.6931 - accuracy: 0.5088
Epoch 5/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6917 - accuracy: 0.5263
Epoch 6/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6925 - accuracy: 0.4971
Epoch 7/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6921 - accuracy: 0.5263
Epoch 8/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6922 - accuracy: 0.5673
Epoch 9/3000
171/171 [==============================] - 0s 12us/step - loss: 0.6926 - accuracy: 0.4912
Epoch 10/3000
171/171 [==============================] - 0s 12us/step - lo

...

Epoch 2625/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3881 - accuracy: 0.8465
Epoch 2626/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3879 - accuracy: 0.8465
Epoch 2627/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3888 - accuracy: 0.8421
Epoch 2628/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3885 - accuracy: 0.8553
Epoch 2629/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3934 - accuracy: 0.8421
Epoch 2630/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3820 - accuracy: 0.8553
Epoch 2631/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3854 - accuracy: 0.8421
Epoch 2632/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3876 - accuracy: 0.8553
Epoch 2633/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3985 - accuracy: 0.8465
Epoch 2634/3000
228/228 [===========================

Epoch 2703/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3881 - accuracy: 0.8596
Epoch 2704/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3856 - accuracy: 0.8421
Epoch 2705/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3871 - accuracy: 0.8509
Epoch 2706/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3808 - accuracy: 0.8465
Epoch 2707/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3888 - accuracy: 0.8421
Epoch 2708/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3876 - accuracy: 0.8465
Epoch 2709/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3854 - accuracy: 0.8465
Epoch 2710/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3920 - accuracy: 0.8640
Epoch 2711/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3945 - accuracy: 0.8596
Epoch 2712/3000
228/228 [=============================

Epoch 2781/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3845 - accuracy: 0.8509
Epoch 2782/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3989 - accuracy: 0.8465
Epoch 2783/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3806 - accuracy: 0.8377
Epoch 2784/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3892 - accuracy: 0.8465
Epoch 2785/3000
228/228 [==============================] - 0s 18us/step - loss: 0.3828 - accuracy: 0.8509
Epoch 2786/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3845 - accuracy: 0.8509
Epoch 2787/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3886 - accuracy: 0.8465
Epoch 2788/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3941 - accuracy: 0.8377
Epoch 2789/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3880 - accuracy: 0.8421
Epoch 2790/3000
228/228 [============================

Epoch 2859/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3888 - accuracy: 0.8509
Epoch 2860/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3872 - accuracy: 0.8509
Epoch 2861/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3839 - accuracy: 0.8684
Epoch 2862/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3784 - accuracy: 0.8509
Epoch 2863/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3843 - accuracy: 0.8509
Epoch 2864/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3843 - accuracy: 0.8553
Epoch 2865/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3889 - accuracy: 0.8596
Epoch 2866/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3991 - accuracy: 0.8509
Epoch 2867/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3889 - accuracy: 0.8596
Epoch 2868/3000
228/228 [=============================

Epoch 2937/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3810 - accuracy: 0.8421
Epoch 2938/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3828 - accuracy: 0.8333
Epoch 2939/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3923 - accuracy: 0.8553
Epoch 2940/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3797 - accuracy: 0.8640
Epoch 2941/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3976 - accuracy: 0.8596
Epoch 2942/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3908 - accuracy: 0.8509
Epoch 2943/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3871 - accuracy: 0.8465
Epoch 2944/3000
228/228 [==============================] - 0s 9us/step - loss: 0.3867 - accuracy: 0.8465
Epoch 2945/3000
228/228 [==============================] - 0s 13us/step - loss: 0.3870 - accuracy: 0.8465
Epoch 2946/3000
228/228 [=============================